In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## About :
1. My Aim here is to introduce some new features 
2. I will also use modelling using LightGBM 
3. This Kernel can be improved further by adding more features and interaction between features 
4. The Modelling itself can be improved by using early stopping and tweaking parameters of Model
5. Any Suggestions for improving kernel is highly encouraged 

## Basic EDA

In [ ]:
train = pd.read_csv("../input/tabular-playground-series-jun-2021/train.csv")
test = pd.read_csv("../input/tabular-playground-series-jun-2021/test.csv")
ss = pd.read_csv("../input/tabular-playground-series-jun-2021/sample_submission.csv")

In [ ]:
train.shape

In [ ]:
train.head()

Observations:
1. We have 200K Rows 
2. We Have 75 features and they all seem to be Numeric
3. The Target is single class to predict but we have provide class probabilities in final submission

In [ ]:
## separate train and target cols
train_cols = ['feature_'+str(i) for i in range(75)]
train_cols[:3]

In [ ]:
target_cols = ['Class_'+str(i) for i in range(1,10)]
target_cols

In [ ]:
## train_Y Stores targets
train_Y = train['target']
train_Y  = pd.DataFrame(train_Y)

In [ ]:
## discard id and train_targets for FE
train = train[train_cols]

In [ ]:
train.head()

In [ ]:
train_Y.head()

## Feature Engineering and Train Test Split

1. I will be using all features provided 
2. In addition I will be using stats like mean , median ,std dev , max , min , kurt , skew etc , rowwise 
3. There is lot of scope of feature enginerring , I am just using starter features here

In [ ]:
train.head()

In [ ]:
train['sum'] = 0 
train['sum'] = train[train_cols].sum(axis = 1)
train['mean'] = 0 
train['mean'] = train[train_cols].mean(axis = 1)
train['median'] = 0 
train['median'] = train[train_cols].median(axis = 1)
train['std'] = 0
train['std'] = train[train_cols].std(axis = 1)
train['mode'] = 0 
train['mode'] = train[train_cols].mode(axis = 1)
train['skew'] = 0 
train['skew'] = train[train_cols].skew(axis = 1)
train['kurt'] = 0
train['kurt'] = train[train_cols].kurtosis(axis =1)
train['max'] = 0
train['max'] = train[train_cols].max(axis =1)
train['min'] = 0 
train['min'] = train[train_cols].min(axis=1)

In [ ]:
train.head()

In [ ]:
## Convert to Categorical 
train_Y['target'] =  train_Y['target'].astype('category').cat.codes

In [ ]:
## Split into train and test
from sklearn.model_selection import train_test_split
temp = pd.concat([train , train_Y] , axis = 1)
X_train, X_test, y_train, y_test = train_test_split(temp[train_cols], temp['target'], test_size=0.33, random_state=42)

In [ ]:
print(X_train.shape , y_train.shape)
print(X_test.shape , y_test.shape)

In [ ]:
X_train.head()

In [ ]:
y_train.head()

## Modelling Using Lightgbm

In [ ]:
import lightgbm as lgb

d_train=lgb.Dataset(X_train, label=y_train)
d_valid=lgb.Dataset(X_test, label=y_test)

In [ ]:
params={}
params['learning_rate']=0.03
params['boosting_type']='gbdt' #GradientBoostingDecisionTree
params['objective']='multiclass' #Multi-class target feature
params['metric']='multi_logloss' #metric for multi-class
params['max_depth']=50
params['num_class']=9 #no.of unique values in the target class not inclusive of the end value
#training the model
clf=lgb.train(params,d_train,
              valid_sets=[d_train, d_valid],
                    verbose_eval=100,
                    num_boost_round=1000,
                    early_stopping_rounds=10)  #training the model on 100 epocs
#prediction on the test dataset


In [ ]:
## Report LogLoss
from sklearn.metrics import mean_squared_error,roc_auc_score,precision_score
from sklearn.metrics import log_loss
log_loss(y_test , clf.predict(X_test))

## Feature Importance

In [ ]:
_ = lgb.plot_importance(clf ,max_num_features = 20)

## Prediction on Test Set

In [ ]:
test.head()

In [ ]:
test['sum'] = 0 
test['sum'] = test[train_cols].sum(axis = 1)
test['mean'] = 0 
test['mean'] = test[train_cols].mean(axis = 1)
test['median'] = 0 
test['median'] = test[train_cols].median(axis = 1)
test['std'] = 0
test['std'] = test[train_cols].std(axis = 1)
test['mode'] = 0 
test['mode'] = test[train_cols].mode(axis = 1)
test['skew'] = 0 
test['skew'] = test[train_cols].skew(axis = 1)
test['kurt'] = 0
test['kurt'] = test[train_cols].kurtosis(axis =1)
test['max'] = 0
test['max'] = test[train_cols].max(axis =1)
test['min'] = 0 
test['min'] = test[train_cols].min(axis=1)

In [ ]:
y_preds  = clf.predict(test[train_cols])

In [ ]:
ss.head()

In [ ]:
ss.loc[:,1:] = y_preds

In [ ]:
ss.head()

In [ ]:
## check 
ss.iloc[: , 1:].sum(axis = 1)

In [ ]:
ss.to_csv("submission.csv" , index = False)